In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd, numpy as np
import os, tqdm 

In [2]:
raw_data_path = "raw/signalmedia-1m/m-articles.jsonl"
df = pd.read_json(raw_data_path, lines=True)

## Preparation Goals
1. Remove sources with only 150 words in content
2. Remove sources with any NaN values
3. Introduce column called "content-cut"
    - Content with only 100 tokens  
4. Split into train & test
    - No need for test yet but more useful to separate early on

### No NaN values, seen below

In [3]:
df.isnull().sum().any()

False

In [4]:
# Completed in increments to not blow up my computer
groups = 10
split_up = np.split(df['content'], groups)
word_counts = []

for group in tqdm.tqdm(split_up):
    lengths = list(map(lambda x : len(x.split()), group))
    word_counts += lengths
df['word-count'] = word_counts

C:\Users\Sam\anaconda3\envs\wmark-pt\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:15<00:00,  1.54s/it]


In [5]:
df_more_than_150 = df[df['word-count'] > 150]

In [9]:
groups = 10
split_up = np.array_split(df_more_than_150['content'], groups)
cut_off = []
ending = []

for group in tqdm.tqdm(split_up):
    cut_off_group = list(map(lambda x : " ".join(x.split()[:100]), group))
    ending_group = list(map(lambda x : " ".join(x.split()[100:]), group))
    cut_off += cut_off_group
    ending += ending_group
df_more_than_150['content-to-sample'] = cut_off
df_more_than_150['content-actual-ending'] = ending 

C:\Users\Sam\anaconda3\envs\wmark-pt\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:37<00:00,  3.72s/it]
C:\Users\Sam\AppData\Local\Temp\ipykernel_2856\3373485034.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_more_than_150['content-to-sample'] = cut_off
C:\Users\Sam\AppData\Local\Temp\ipykernel_2856\3373485034.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [10]:
df_more_than_150 = df_more_than_150[['id', 'title', 'source', 'content', 'word-count', 'content-actual-ending', 'content-to-sample', 'published']]

In [11]:
train_df, test_df = train_test_split(df_more_than_150, test_size=0.2)

In [12]:
train_df.to_csv("prepared/train/training_untokenized.csv")
test_df.to_csv("prepared/test/test_untokenized.csv")